In [16]:
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support.wait import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.action_chains import ActionChains
from bs4 import BeautifulSoup
import pandas as pd
import openpyxl
import os
import time
import math
import webbrowser
import numpy as np
import re
import sys


# 찾을 품목 검색어 입력하기
# prod_text = input('검색할 상품명을 입력하세요 : ')
collect_cnt = int(input("몇 건의 상품을 검색하겠습니까?"))
f_dir = input("저장할 파일경로를 입력하세요(ex = C:\\temp\\won.txt)")
if f_dir == '':
    f_dir = "c:\\temp\\" + 'cordishop'+".txt"
    print(f_dir)
# cnt = int(input('몇 건의 데이터를 수집하시겠습니까?'))
#musinsa 사이트 열기
chrome_path = "C:\Temp\chromedriver.exe"

options = webdriver.ChromeOptions()
options.headless = True
#지정한 user-agent로 설정합니다.
user_agent = "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/107.0.0.0 Safari/537.36"
options.add_argument("no-sandbox")
options.add_argument("disable-gpu")
options.add_argument("--lang=ko_KR")
options.add_argument('user-agent=' + user_agent)
driver = webdriver.Chrome(chrome_path,options=options)
driver.execute_cdp_cmd("Page.addScriptToEvaluateOnNewDocument", 
{"source": """ Object.defineProperty(navigator, 'webdriver', { get: () => undefined }) """})
url = "https://www.musinsa.com/app/"
driver.get(url)
driver.set_window_size(1920,1080)
driver.maximize_window()

#남성 태그 클릭
WebDriverWait(driver, 5).until(EC.element_to_be_clickable((By.XPATH, "/html/body/div[3]/div[3]/div[9]/button[2]"))).click()
WebDriverWait(driver, 5).until(EC.element_to_be_clickable((By.XPATH, "/html/body/div[2]/div/div[2]/div/div[1]/div[2]/ul/li[3]"))).click()
time.sleep(2)

#코디맵 버튼

WebDriverWait(driver, 8).until(EC.element_to_be_clickable((By.XPATH, "/html/body/div[2]/div/div[2]/div/div[1]/div[2]/ul/li[4]/a"))).click()
time.sleep(2)
WebDriverWait(driver, 8).until(EC.element_to_be_clickable((By.XPATH, "/html/body/div[3]/div[3]/form/div[2]/div[1]/div/a[2]"))).click()
time.sleep(2)
#조회수 button
WebDriverWait(driver, 5).until(EC.element_to_be_clickable((By.XPATH, "/html/body/div[3]/div[3]/form/div[4]/div/div[1]/a[2]/span"))).click()

html= driver.page_source
soup= BeautifulSoup(html,'html.parser')    
last_page = int(soup.find("span", attrs={"class":"totalPagingNum"}).get_text().strip())
total_stylenums = soup.find('span',attrs = {'class':'counter'}).get_text().strip()
total_stylenum = int(re.sub(r'[^0-9]', '', total_stylenums))
page_no = []
#화살표 버튼 누르는 번호 리스트 화
for i in range(11,last_page+1) :
     if i%10 == 1:
        page_no.append(i)
#검색 할 페이지 수 정하기            
collect_page_cnt = math.ceil(collect_cnt/60)

if collect_page_cnt>last_page:
    print("조회 가능한 상품 수를 초과하셨습니다")
    sys.exit()
cordi_style_list=[]
go_views=[]
urls=[]
for k in range(1,collect_page_cnt+1):
    url = "https://www.musinsa.com/app/codimap/lists?style_type=&tag_no=&brand=&display_cnt=60&list_kind=big&sort=view_cnt&page="+f'{k}'
    driver.get(url)
    time.sleep(2)
    html_2= driver.page_source
    soup_2= BeautifulSoup(html_2,'html.parser')
    
#     크롤링 범위 가져오기
    item_boxs = soup_2.find("div", attrs={"class":"boxed-list-wrapper"})
    cordi_styles = item_boxs.find_all('span',attrs = {'class':'style-list-information__text'})
    items_box = item_boxs.find_all("div", attrs={"class":"style-list-item__thumbnail"})
    for cordi_style in cordi_styles:
        style=cordi_style.get_text().strip()
        cordi_style_list.append(style)
        if collect_cnt<=len(cordi_style_list):
            break
            
    for item_box1 in items_box:
        item_goview = item_box1.find('a')['onclick']            
        numbers = re.sub(r'[^0-9]', '',item_goview)
        go_views.append(numbers)
        
        
    for go_view in go_views:
        style_url = ('https://www.musinsa.com/app/codimap/views/'+go_view+'?style_type=&tag_no=&brand=&display_cnt'
                     +'=60&list_kind=big&sort=date&page='+str(k))
        urls.append(style_url)
    

item_no=1
extract_data=[]
extract_hashtag=[] 
for c in range(0,collect_cnt):

    print(item_no," ",urls[c])
    driver.get(urls[c])
    time.sleep(1)
    html_3 = driver.page_source
    soup_3 = BeautifulSoup(html_3,"html.parser")
    
    try:
        hashtags=soup_3.find_all('a',attrs = {'class':'ui-tag-list__item'})
        
        cordi_details = soup_3.find('p',attrs = {'class':'styling_date'}).get_text()
        details = cordi_details.split('|')
        extract_data.append([])
        
        for detail in details:
            extract_data[c].append(int(re.sub(r'[^0-9]', '', detail)))
        for hashtag in hashtags:           
            new_hashtag= hashtag.get_text().strip()
            extract_data[c].append(re.sub(r"[^\uAC00-\uD7A30-9a-zA-Z\s]", "", new_hashtag))
            
    except:
        pass
    else:
        item_no+=1
   

몇 건의 상품을 검색하겠습니까?100
저장할 파일경로를 입력하세요(ex = C:\temp\won.txt)
c:\temp\cordishop.txt


C:\Users\won\AppData\Local\Temp\ipykernel_21324\2288519978.py:40: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver = webdriver.Chrome(chrome_path,options=options)


1   https://www.musinsa.com/app/codimap/views/5053?style_type=&tag_no=&brand=&display_cnt=60&list_kind=big&sort=date&page=1
2   https://www.musinsa.com/app/codimap/views/1701?style_type=&tag_no=&brand=&display_cnt=60&list_kind=big&sort=date&page=1
3   https://www.musinsa.com/app/codimap/views/1969?style_type=&tag_no=&brand=&display_cnt=60&list_kind=big&sort=date&page=1
4   https://www.musinsa.com/app/codimap/views/326?style_type=&tag_no=&brand=&display_cnt=60&list_kind=big&sort=date&page=1
5   https://www.musinsa.com/app/codimap/views/2979?style_type=&tag_no=&brand=&display_cnt=60&list_kind=big&sort=date&page=1
6   https://www.musinsa.com/app/codimap/views/1974?style_type=&tag_no=&brand=&display_cnt=60&list_kind=big&sort=date&page=1
7   https://www.musinsa.com/app/codimap/views/1169?style_type=&tag_no=&brand=&display_cnt=60&list_kind=big&sort=date&page=1
8   https://www.musinsa.com/app/codimap/views/2442?style_type=&tag_no=&brand=&display_cnt=60&list_kind=big&sort=date&page=1
9   https

67   https://www.musinsa.com/app/codimap/views/1169?style_type=&tag_no=&brand=&display_cnt=60&list_kind=big&sort=date&page=2
68   https://www.musinsa.com/app/codimap/views/2442?style_type=&tag_no=&brand=&display_cnt=60&list_kind=big&sort=date&page=2
69   https://www.musinsa.com/app/codimap/views/283?style_type=&tag_no=&brand=&display_cnt=60&list_kind=big&sort=date&page=2
70   https://www.musinsa.com/app/codimap/views/6738?style_type=&tag_no=&brand=&display_cnt=60&list_kind=big&sort=date&page=2
71   https://www.musinsa.com/app/codimap/views/6174?style_type=&tag_no=&brand=&display_cnt=60&list_kind=big&sort=date&page=2
72   https://www.musinsa.com/app/codimap/views/2059?style_type=&tag_no=&brand=&display_cnt=60&list_kind=big&sort=date&page=2
73   https://www.musinsa.com/app/codimap/views/3014?style_type=&tag_no=&brand=&display_cnt=60&list_kind=big&sort=date&page=2
74   https://www.musinsa.com/app/codimap/views/7449?style_type=&tag_no=&brand=&display_cnt=60&list_kind=big&sort=date&page=2
7

In [1]:

df = pd.DataFrame(extract_data)
df

NameError: name 'pd' is not defined

In [17]:
from collections import Counter


#1~100위 스타일 분포
style_cate=np.array(list(set(cordi_style_list)))
array_style = np.array(cordi_style_list)
style_dict=Counter(array_style)
list_key=[]
list_value = []
for key,value in style_dict.items():
    list_key.append(key)
    list_value.append(value)

n = time.localtime()
s = f'{n.tm_year}-{n.tm_mon}-{n.tm_mday}-{n.tm_sec}'
fc_name = s+'styletrend'+'.csv'
df_styletrend= pd.DataFrame(list_value,index=list_key,columns=['개수'])
df_styletrend.to_csv(fc_name,encoding='cp949') 

In [18]:
list_key

['캐주얼', '스트릿', '댄디', '포멀', '스포츠', '아메리칸 캐주얼']

In [21]:
n = time.localtime()
s = f'{n.tm_year}-{n.tm_mon}-{n.tm_mday}-{n.tm_sec}'
fc_name = f_dir+s+'해시태그 by 스타일'+'.csv'
df = pd.DataFrame(extract_data)
df.insert(1,'코디',cordi_style_list)
df
#\
# df.to_csv(fc_name,encoding='cp949') 

,0,코디,1,2,3,4,5,6,7,8,...,15,16,17,18,19,20,21,22,23,24
0,20210730,캐주얼,35740,16,로고플레이,편안한,원마일웨어,와이드핏,여름,캐주얼,...,반소매 티셔츠,안경,트레이닝조거 팬츠,캔버스단화,캡야구 모자,None,None,None,None,None
1,20210203,스트릿,33392,19,봄,스트릿,제로,월간,라퍼지스토어,키치오브제,...,캔버스단화,니트스웨터,캡야구 모자,None,None,None,None,None,None,None
2,20210223,캐주얼,31101,42,봄,캐주얼,라퍼지스토어,다이아몬드 레이라,나이키,엠프렌즈,...,패션스니커즈화,케이스슬리브,캡야구 모자,레더라이더스 재킷,맨투맨스웨트셔츠,None,None,None,None,None
3,20201029,스트릿,27565,4,가을,스트릿,제로,쇼브오프,라퍼지스토어,크리틱,...,None,None,None,None,None,None,None,None,None,None
4,20210430,댄디,27235,2,봄,댄디,후러브스아트,발렌티노 루디,오니츠카타이거,예일,...,반소매 티셔츠,메신저크로스 백,None,None,None,None,None,None,None,None
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
95,20210702,댄디,13690,42,톤온톤,뉴트럴톤,캠퍼스,피크닉,와이드핏,오버핏,...,예일,반원 아틀리에,토트백,잡지무크지,캡야구 모자,캔버스단화,반소매 티셔츠,None,None,None
96,20211110,댄디,13633,편안한,원마일웨어,로고플레이,한파,가을,캐주얼,골라 클래식,...,케이스슬리브,트레이닝조거 팬츠,숏패딩숏헤비 아우터,메신저크로스 백,None,None,None,None,None,None
97,20201207,댄디,13623,겨울,캐주얼,아식스,크럼프,예일,아웃스탠딩,패션스니커즈화,...,None,None,None,None,None,None,None,None,None,None
98,20220328,캐주얼,13492,1,모노톤,데이트,레더,에코,셀럽템,오버핏,...,셔츠블라우스,구두,None,None,None,None,None,None,None,None
